<a href="https://colab.research.google.com/github/popolee0513/CS50/blob/main/pyspark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/spark')
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

In [ ]:
sc=SparkSession.builder.appName("PysparkSQL").config("spark.sql.shuffle.partitions","50").config("spark.driver.maxResultSize","5g")\
                      .config("spark.sql.execution.arrow.enabled", "true")\
                      .config("spark.sql.legacy.timeParserPolicy","LEGACY").getOrCreate()

In [ ]:
df = sc.read.csv('data.csv',header=True)
df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
df.dtypes

[('InvoiceNo', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'string'),
 ('InvoiceDate', 'string'),
 ('UnitPrice', 'string'),
 ('CustomerID', 'string'),
 ('Country', 'string')]

In [ ]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
print(df.count())
print(df.rdd.getNumPartitions())
df = df.repartition(10)
print(df.rdd.getNumPartitions())
###  check out the distribution of records in a partition 
print(df.rdd.glom().map(len).collect())

541909
2
10
[54191, 54191, 54191, 54191, 54191, 54190, 54191, 54191, 54191, 54191]


In [ ]:
col_org_name = df.columns

In [ ]:
### rename all columns
df = df.toDF(*['1','2','3','4','5','6','7','8'])
df.show(1)

+------+------+--------------------+---+--------------+----+-----+--------------+
|     1|     2|                   3|  4|             5|   6|    7|             8|
+------+------+--------------------+---+--------------+----+-----+--------------+
|562031|90039D|FIRE POLISHED GLA...|  2|8/1/2011 17:37|3.75|16984|United Kingdom|
+------+------+--------------------+---+--------------+----+-----+--------------+
only showing top 1 row



In [ ]:
df = df.toDF(*col_org_name)

In [ ]:
### rename single column name
df = df.withColumnRenamed("CustomerID","UserID")
df.show(3)

+---------+---------+--------------------+--------+---------------+---------+------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|UserID|       Country|
+---------+---------+--------------------+--------+---------------+---------+------+--------------+
|   562031|   90039D|FIRE POLISHED GLA...|       2| 8/1/2011 17:37|     3.75| 16984|United Kingdom|
|   558371|    23205|CHARLOTTE BAG VIN...|      16|6/28/2011 15:44|     2.46|  null|United Kingdom|
|   547042|    21289|LARGE STRIPES CHO...|       2|3/20/2011 11:00|     1.25|  null|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+------+--------------+
only showing top 3 rows



In [ ]:
user =df.select("UserID")
user.show(3)

+------+
|UserID|
+------+
| 17908|
| 17908|
| 17908|
+------+
only showing top 3 rows



In [ ]:
user_country = df.select("UserID","Country")
user_country.show(3)

+------+--------------+
|UserID|       Country|
+------+--------------+
| 13883|United Kingdom|
| 13883|United Kingdom|
| 13883|United Kingdom|
+------+--------------+
only showing top 3 rows



In [ ]:
df = df.filter(df.UnitPrice>0)
df.show(3)

+---------+---------+--------------------+--------+---------------+---------+------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|UserID|       Country|
+---------+---------+--------------------+--------+---------------+---------+------+--------------+
|   548194|    22766| PHOTO FRAME CORNICE|       1|3/29/2011 15:30|     5.79|  null|United Kingdom|
|   562113|    22424|ENAMEL BREAD BIN ...|       1| 8/2/2011 16:11|    24.96|  null|United Kingdom|
|   550351|    82482|WOODEN PICTURE FR...|       6|4/18/2011 10:20|     2.55| 15491|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+------+--------------+
only showing top 3 rows



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType, StringType, BooleanType

df = df.withColumn('UnitPrice', F.col('UnitPrice').cast(DoubleType()))
df = df.withColumn('Quantity', F.col('Quantity').cast(IntegerType()))
df = df.withColumn('InvoiceDate',to_date(col("InvoiceDate"), "MM/dd/yyyy HH:mm"))
df = df.sort(F.desc("UnitPrice"))
df.show(5)

+---------+---------+-----------+--------+-----------+---------+------+--------------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|
+---------+---------+-----------+--------+-----------+---------+------+--------------+
|  C556445|        M|     Manual|      -1| 2011-06-10|  38970.0| 15098|United Kingdom|
|  C580605|AMAZONFEE| AMAZON FEE|      -1| 2011-12-05| 17836.46|  null|United Kingdom|
|  C540117|AMAZONFEE| AMAZON FEE|      -1| 2011-01-05| 16888.02|  null|United Kingdom|
|  C540118|AMAZONFEE| AMAZON FEE|      -1| 2011-01-05| 16453.71|  null|United Kingdom|
|  C537630|AMAZONFEE| AMAZON FEE|      -1| 2010-12-07| 13541.33|  null|United Kingdom|
+---------+---------+-----------+--------+-----------+---------+------+--------------+
only showing top 5 rows



In [ ]:
df = df.withColumn("Amount", F.col("UnitPrice")*F.col("Quantity"))
df.show(3)

+---------+---------+-----------+--------+-----------+---------+------+--------------+---------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|   Amount|
+---------+---------+-----------+--------+-----------+---------+------+--------------+---------+
|  C556445|        M|     Manual|      -1| 2011-06-10|  38970.0| 15098|United Kingdom| -38970.0|
|  C580605|AMAZONFEE| AMAZON FEE|      -1| 2011-12-05| 17836.46|  null|United Kingdom|-17836.46|
|  C540117|AMAZONFEE| AMAZON FEE|      -1| 2011-01-05| 16888.02|  null|United Kingdom|-16888.02|
+---------+---------+-----------+--------+-----------+---------+------+--------------+---------+
only showing top 3 rows



In [ ]:
total_amount =df.groupBy(["UserID"]).agg(
    F.sum("Amount").alias("TotalAmount")
    ).sort(F.desc("TotalAmount")).withColumn("TotalAmount", F.round("TotalAmount", 2)).filter("UserID IS NOT NULL")
total_amount.show(5)

+------+-----------+
|UserID|TotalAmount|
+------+-----------+
| 18102|  254300.09|
| 14646|  240630.51|
| 17450|  185167.97|
| 14911|   114588.8|
| 12415|  109177.41|
+------+-----------+
only showing top 5 rows



In [ ]:
TotalCustomer = df.groupBy(["Country"]).agg(
    F.approx_count_distinct(col('UserID')).alias("TotalCustomer")
    ).sort(F.desc("TotalCustomer"))
TotalCustomer.show(10)

+--------------+-------------+
|       Country|TotalCustomer|
+--------------+-------------+
|United Kingdom|         3653|
|       Germany|           92|
|        France|           88|
|         Spain|           32|
|       Belgium|           26|
|   Switzerland|           21|
|      Portugal|           19|
|         Italy|           15|
|       Finland|           12|
|       Austria|           11|
+--------------+-------------+
only showing top 10 rows



In [ ]:
top_5 = TotalCustomer.limit(5)
top_5 = top_5.rdd.map(lambda x:x[0]).collect()
print(top_5)

['United Kingdom', 'Germany', 'France', 'Spain', 'Belgium']


In [ ]:
from pyspark.sql.window import Window

In [ ]:
window_spec = Window.partitionBy(df['UserID'])
df = df.withColumn('Cumulative Amount', F.sum(col('Amount')).over(window_spec))
df.show(5)

+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|  Amount| Cumulative Amount|
+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+
|  C541433|    23166|MEDIUM CERAMIC TO...|  -74215| 2011-01-18|     1.04| 12346|United Kingdom|-77183.6|               0.0|
|   541431|    23166|MEDIUM CERAMIC TO...|   74215| 2011-01-18|     1.04| 12346|United Kingdom| 77183.6|               0.0|
|   542237|    22423|REGENCY CAKESTAND...|       3| 2011-01-26|    12.75| 12347|       Iceland|   38.25|3764.4800000000023|
|   573511|    22423|REGENCY CAKESTAND...|       6| 2011-10-31|    12.75| 12347|       Iceland|    76.5|3764.4800000000023|
|   549222|    22423|REGENCY CAKESTAND...|       3| 2011-04-07|    12.75| 12347|       Iceland|   38.25|3764.4800000000023|
+-------

In [ ]:
### remove StockCode
new_df = df.drop("StockCode")

In [ ]:
new_df.show(3)

+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+
|InvoiceNo|         Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|  Amount| Cumulative Amount|
+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+
|   541431|MEDIUM CERAMIC TO...|   74215| 2011-01-18|     1.04| 12346|United Kingdom| 77183.6|               0.0|
|  C541433|MEDIUM CERAMIC TO...|  -74215| 2011-01-18|     1.04| 12346|United Kingdom|-77183.6|               0.0|
|   562032|REGENCY CAKESTAND...|       3| 2011-08-02|    12.75| 12347|       Iceland|   38.25|3764.4800000000023|
+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+
only showing top 3 rows



In [ ]:
import re
from pyspark.sql.functions import udf
def check_contain_alphabet(col):
    res = bool(re.match('[a-zA-Z]',col))
    if res:
      return True
    else:
      return False
check_contain_alphabet = F.udf(check_contain_alphabet, BooleanType())
df = df.withColumn("check_contain_alphabet", check_contain_alphabet("StockCode"))
df.show(5)

+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+----------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|  Amount| Cumulative Amount|check_contain_alphabet|
+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+----------------------+
|  C541433|    23166|MEDIUM CERAMIC TO...|  -74215| 2011-01-18|     1.04| 12346|United Kingdom|-77183.6|               0.0|                 false|
|   541431|    23166|MEDIUM CERAMIC TO...|   74215| 2011-01-18|     1.04| 12346|United Kingdom| 77183.6|               0.0|                 false|
|   542237|    22423|REGENCY CAKESTAND...|       3| 2011-01-26|    12.75| 12347|       Iceland|   38.25|3764.4800000000023|                 false|
|   573511|    22423|REGENCY CAKESTAND...|       6| 2011-10-31|    12.75| 12347|       Iceland|    76.5|3764.480000000

In [ ]:
df.filter(df["check_contain_alphabet"]==True).show(5)

+---------+---------+-----------+--------+-----------+---------+------+-------+------+------------------+----------------------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|UserID|Country|Amount| Cumulative Amount|check_contain_alphabet|
+---------+---------+-----------+--------+-----------+---------+------+-------+------+------------------+----------------------+
|   541998|     POST|    POSTAGE|       1| 2011-01-25|     40.0| 12348|Finland|  40.0|            1114.0|                  true|
|   548955|     POST|    POSTAGE|       1| 2011-04-05|     40.0| 12348|Finland|  40.0|            1114.0|                  true|
|   539318|     POST|    POSTAGE|       6| 2010-12-16|     40.0| 12348|Finland| 240.0|            1114.0|                  true|
|   568172|     POST|    POSTAGE|       1| 2011-09-25|     40.0| 12348|Finland|  40.0|            1114.0|                  true|
|   543037|     POST|    POSTAGE|       1| 2011-02-02|     40.0| 12350| Norway|  40.0|295.3000000

In [ ]:
df = df.withColumn("check_contain_alphabet",when(df.check_contain_alphabet == True, 1).otherwise(0))
df.show(5)

+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+----------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|UserID|       Country|  Amount| Cumulative Amount|check_contain_alphabet|
+---------+---------+--------------------+--------+-----------+---------+------+--------------+--------+------------------+----------------------+
|  C541433|    23166|MEDIUM CERAMIC TO...|  -74215| 2011-01-18|     1.04| 12346|United Kingdom|-77183.6|               0.0|                     0|
|   541431|    23166|MEDIUM CERAMIC TO...|   74215| 2011-01-18|     1.04| 12346|United Kingdom| 77183.6|               0.0|                     0|
|   542237|    22423|REGENCY CAKESTAND...|       3| 2011-01-26|    12.75| 12347|       Iceland|   38.25|3764.4800000000023|                     0|
|   573511|    22423|REGENCY CAKESTAND...|       6| 2011-10-31|    12.75| 12347|       Iceland|    76.5|3764.480000000

In [ ]:
pivot = df[df["Country"].isin(top_5)].groupBy('InvoiceDate').pivot('Country').agg(F.round(F.sum('Amount'),3).alias('TotalAmount')).na.fill(value=0)
for col in pivot.columns[1:]:
   pivot= pivot.withColumn(col, when(pivot[col]<0,0).otherwise(pivot[col]))

pivot.show()

+-----------+-------+-------+-------+------+--------------+
|InvoiceDate|Belgium| France|Germany| Spain|United Kingdom|
+-----------+-------+-------+-------+------+--------------+
| 2011-11-29|  281.9|1274.16|1290.36|136.39|      60434.59|
| 2011-06-03|    0.0|    0.0|   68.1| 264.4|      13550.75|
| 2011-11-02|    0.0| 601.24| 110.03|   0.0|      32920.52|
| 2011-10-03|    0.0| 346.57|    0.0|   0.0|      59725.44|
| 2011-10-07|    0.0|  474.6|2474.51|340.54|       37415.1|
| 2011-05-06| 367.48|    0.0| 138.98| 522.6|      26360.16|
| 2011-08-14|    0.0|  46.65|    0.0|   0.0|       4777.16|
| 2011-01-30|    0.0|    0.0|    0.0|   0.0|       5693.28|
| 2011-07-27|1052.77| 163.35|  394.8|   0.0|      17234.54|
| 2011-04-10| 469.69|    0.0|    0.0|   0.0|       7550.94|
| 2011-07-12|    0.0|1082.15| 373.92|   0.0|      20369.11|
| 2011-03-09|    0.0|    0.0|    0.0|   0.0|      16606.04|
| 2011-10-31| 422.05| 330.76| 631.13|   0.0|      41233.78|
| 2011-07-07| 281.65|    0.0|1086.41| 35

In [ ]:
# "stack(<cnt of columns you want to put in one column>, 'firstcolname', firstcolname , 'secondcolname' ,secondcolname ......) as (Type, Value)"
pivot = pivot.withColumnRenamed("United Kingdom","United_Kingdom")
expression = ""
cnt=0
for column in pivot.columns:
    if column!='InvoiceDate':
        cnt +=1
        expression += f"'{column}' , {column},"
expression = f"stack({cnt}, {expression[:-1]}) as (Country,TotalSales)"

unpivot = pivot.select('InvoiceDate',F.expr(expression))
unpivot.show()

+-----------+--------------+----------+
|InvoiceDate|       Country|TotalSales|
+-----------+--------------+----------+
| 2011-11-29|       Belgium|     281.9|
| 2011-11-29|        France|   1274.16|
| 2011-11-29|       Germany|   1290.36|
| 2011-11-29|         Spain|    136.39|
| 2011-11-29|United_Kingdom|  60434.59|
| 2011-06-03|       Belgium|       0.0|
| 2011-06-03|        France|       0.0|
| 2011-06-03|       Germany|      68.1|
| 2011-06-03|         Spain|     264.4|
| 2011-06-03|United_Kingdom|  13550.75|
| 2011-11-02|       Belgium|       0.0|
| 2011-11-02|        France|    601.24|
| 2011-11-02|       Germany|    110.03|
| 2011-11-02|         Spain|       0.0|
| 2011-11-02|United_Kingdom|  32920.52|
| 2011-10-03|       Belgium|       0.0|
| 2011-10-03|        France|    346.57|
| 2011-10-03|       Germany|       0.0|
| 2011-10-03|         Spain|       0.0|
| 2011-10-03|United_Kingdom|  59725.44|
+-----------+--------------+----------+
only showing top 20 rows

